# Reading Bronhouderportaal BRO data

This notebook introduces how to use the `hydropandas` package to read, visualise and analyse meta data of newly installed groundwater wells. These meta data is to be submitted to Bronhouderportaal BRO afterwards.

## <a id=top></a>Notebook contents

1. [Read ObsCollection](#readingOC)
2. [Visualise ObsCollection](#visualize)
3. [Analyse ObsCollection](#analyse)

In [ ]:
import hydropandas as hpd

import logging
from IPython.display import HTML

import pandas as pd


In [ ]:
hpd.util.get_color_logger("INFO")

## Read ObsCollection<a id=readingOC></a>

An `ObsCollection` is created for the multiple monitoring wells.

In [ ]:
dirname = "data/bronhouderportaal-bro"
oc = hpd.read_bronhouderportaal_bro(dirname, full_meta=True, add_to_df=True)
oc

## Visualize ObsCollection<a id=visualize></a>

Visualize the `ObsCollection`.

In [ ]:
# TO DO: interactive_map() does not support locations without observations
# oc.plots.interactive_map()


In [ ]:
# TO DO: section_plot() now hacked to support locations without observations, but observation axis is still plotted (and empty)
oc.plots.section_plot(plot_obs=False)


## Analyse ObsCollection<a id=analyse></a>

Visualize the `ObsCollection`.

First step is to check which columns have unique values for all wells. E.g. the owner should be the same for all wells. That requires that we drop the `obs` column, beceause `pd.nunique` cannot deal with that specific HydroPandas column-type.

In [ ]:
oc_temp = oc.copy().drop(["obs"], axis=1)
oc_unique = oc_temp.iloc[0][oc_temp.columns[oc_temp.nunique() <= 1]]
oc_unique

In [ ]:
oc_non_unique = oc[oc.columns.drop(oc_temp.columns[oc_temp.nunique() <= 1])]
oc_non_unique


In [ ]:
# get statistics
oc_non_unique.describe()


Check the usage of `tube_nr`. Has tube number one the lowest `screen_bottom` and lowest `screen_top`?

In [ ]:
lst_lowest_tube = []
for monitoring_well in oc.monitoring_well.unique():
    oc_mw = oc.loc[oc.monitoring_well == monitoring_well]

    lowest_screen_bottom_tube_nr = oc_mw.loc[
        oc_mw.screen_bottom == oc_mw.screen_bottom.min(), "tube_nr"
    ].values[0]

    lowest_screen_top_tube_nr = oc_mw.loc[
        oc_mw.screen_top == oc_mw.screen_top.min(), "tube_nr"
    ].values[0]

    lst_lowest_tube.append(
        [monitoring_well, lowest_screen_bottom_tube_nr, lowest_screen_top_tube_nr]
    )

df_lowest_tube = pd.DataFrame(
    lst_lowest_tube,
    columns=[
        "monitoring_well",
        "lowest_screen_bottom_tube_nr",
        "lowest_screen_top_tube_nr",
    ],
).set_index("monitoring_well")

df_lowest_tube

## Upload to Bronhouderportaal BRO

Upload the XML-files to Bronhouderportaal BRO via the website.